# Machine Learning in the SDSS catalog: 
## Photometric Redshift Estimation and Quasar Classification

Modern wide field surveys are generating very large databases of automatically measured objects, whose error properties may not be well understood. Fast machine learning algorithms have been proved to be very useful in such a regime.

Let's investigate the SDSS photometric object catalog, and look for machine learning solutions to the following two problems:

1. Estimating the redshifts of quasars from their photometry (regression)

1. Selecting quasars from a background of stars and galaxies (classification)

## Data Aquisition

From the SDSS Sky Server we've downloaded two types of photometry (aperature and petrosian), corrected for extinction, for a number of sources with redshifts. Here's the SQL for an example query, that gets us 10000 example quasars:

<font color="blue">
 <pre>SELECT *,dered_u - mag_u AS diff_u, dered_g - mag_g AS diff_g, dered_r - mag_r AS diff_g, dered_i - mag_i AS diff_i, dered_z - mag_z AS diff_z from
(SELECT top 10000
objid, ra, dec, dered_u,dered_g,dered_r,dered_i,dered_z,psfmag_u-extinction_u AS mag_u,
psfmag_g-extinction_g AS mag_g, psfmag_r-extinction_r AS mag_r, psfmag_i-extinction_i AS mag_i,psfmag_z-extinction_z AS mag_z,z AS spec_z,dered_u - dered_g AS u_g_color, 
dered_g - dered_r AS g_r_color,dered_r - dered_i AS r_i_color,dered_i - dered_z AS i_z_color,class
FROM SpecPhoto 
WHERE 
 (class = 'QSO')
 ) as sp
 </pre>
</font>

We've got 1000 stars and 1000 galaxies as well, and saved them for convenience.

In [ ]:
# For pretty plotting
!pip install --upgrade seaborn

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
%pylab inline
import seaborn as sns
sns.set()
import copy

## Photometric Redshift Estimation

* This is a regression problem, to be able to predict the redshift response variable given a number of photometric measurement "features".

In [ ]:
qsos = pd.read_csv("data/qso10000.csv",index_col=0,usecols=["objid","dered_r","spec_z","u_g_color",\
                                                "g_r_color","r_i_color","i_z_color","diff_u",\
                                                "diff_g1","diff_i","diff_z"])

# Clean out extreme colors and bad magnitudes:
qsos = qsos[(qsos["dered_r"] > -9999) & (qsos["g_r_color"] > -10) & (qsos["g_r_color"] < 10)]


# Response variables: redshift
qso_redshifts = qsos["spec_z"]

# Features or attributes: photometric measurements
qso_features = copy.copy(qsos)
del qso_features["spec_z"]
qso_features.head()

In [ ]:
bins =  hist(qso_redshifts.values,bins=100) ; xlabel("redshift") ; ylabel("N")

Pretty clearly a big cut at around $z=2$.

Let's plot all the features, colored by the target redshift, to look for structure.

In [ ]:
import matplotlib as mpl
import matplotlib.cm as cm

# Truncate the color at z=2.5 just to keep some contrast.
norm = mpl.colors.Normalize(vmin=min(qso_redshifts.values), vmax=2.5)
cmap = cm.jet_r
m = cm.ScalarMappable(norm=norm, cmap=cmap)

# Plot everything against everything else:
rez = pd.scatter_matrix(qso_features[0:2000],alpha=0.2,figsize=[15,15],color=m.to_rgba(qso_redshifts.values))

In [ ]:
X = qso_features.values  # 9-d feature space
y = qso_redshifts.values # redshifts

In [ ]:
print("Design matrix shape =", X.shape)
print("Response variable vector shape =", y.shape)

In [ ]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

** Linear Regression **

In [ ]:
from sklearn import linear_model
linear = linear_model.LinearRegression()

# Fit the model, using all the attributes:
linear.fit(X_train, y_train)

# Do the prediction on the test data:
y_lr_pred = linear.predict(X_test)

# How well did we do?
from sklearn.metrics import mean_squared_error
mse = np.sqrt(mean_squared_error(y_test,y_lr_pred))
print ("Linear regression: MSE = ",mse)
print("R2 score =",linear.score(X_test, y_test))

In [ ]:
plot(y_test,y_lr_pred - y_test,'o',alpha=0.2)
title("Linear Regression Residuals - MSE = %.1f" % mse)
xlabel("Spectroscopic Redshift")
ylabel("Residual")
hlines(0,min(y_test),max(y_test),color="red")

Just how bad is this? Here's the MSE from guessing the *average redshift of the training set* for all new objects:

In [ ]:
print("Naive MSE", ((1./len(y_train))*(y_train - y_train.mean())**2).sum())

In [ ]:
mean_squared_error?

** *k*-Nearest Neighbor (KNN) Regression **

["Regression based on k-nearest neighbors. The target is predicted by local interpolation of the targets associated of the nearest neighbors in the training set."](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)

In [ ]:
from sklearn import neighbors
from sklearn import preprocessing

X_scaled = preprocessing.scale(X) # Many methods work better on scaled X.

KNN = neighbors.KNeighborsRegressor(5)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)

KNN.fit(X_train,y_train)

In [ ]:
y_knn_pred = KNN.predict(X_test)
mse = mean_squared_error(y_test,y_knn_pred)
print("MSE (KNN) =", mse)
print("R2 score (KNN) =",KNN.score(X_test, y_test))

In [ ]:
plot(y_test, y_knn_pred - y_test,'o',alpha=0.2)
title("k-NN Residuals - MSE = %.2f" % mse)
xlabel("Spectroscopic Redshift")
ylabel("Residual")
hlines(0,min(y_test),max(y_test),color="red")

### Tuning the KNN Model

* Let's do a grid search over the control parameters of the KNN model, to see how good we can make our predictions.

* We can see our options in the model `repr`:

> KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski', metric_params=None, n_neighbors=5, p=2, weights='uniform')

* Let's first make a "validation curve" to investigate one parameter: the number of nearest neighbors averaged over.

In [ ]:
# We'll vary the number of neighbors used:
param_name = "n_neighbors"
param_range = np.array([1,2,4,8,16,32,64])

# And we'll need a cv iterator:
from sklearn.cross_validation import ShuffleSplit
shuffle_split = ShuffleSplit(len(X), 10, test_size=0.4)

# Compute our cv scores for a range of the no. of neighbors:
from sklearn.learning_curve import validation_curve
training_scores, validation_scores = validation_curve(KNN, X_scaled, y,
                                                      param_name=param_name,
                                                      param_range=param_range, 
                                                      cv=shuffle_split, scoring='r2')

In [ ]:
def plot_validation_curve(param_name,parameter_values, training_scores, validation_scores):
    training_scores_mean = np.mean(training_scores, axis=1)
    training_scores_std = np.std(training_scores, axis=1)
    validation_scores_mean = np.mean(validation_scores, axis=1)
    validation_scores_std = np.std(validation_scores, axis=1)

    plt.fill_between(parameter_values, training_scores_mean - training_scores_std,
                     training_scores_mean + training_scores_std, alpha=0.1, color="r")
    plt.fill_between(parameter_values, validation_scores_mean - validation_scores_std,
                     validation_scores_mean + validation_scores_std, alpha=0.1, color="g")
    plt.plot(parameter_values, training_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(parameter_values, validation_scores_mean, 'o-', color="g",
             label="Cross-validation score")
    plt.ylim(validation_scores_mean.min() - .1, training_scores_mean.max() + .1)
    plt.xlabel(param_name)
    plt.legend(loc="best")

In [ ]:
plot_validation_curve(param_name, param_range, training_scores, validation_scores)

#### Question:

Can you explain the shapes of these two curves? Talk to your neighbor for a few minutes, and be prepared to suggest reasons for a) the rise and fall of the cross validation score and b) the monotonic decrease in training score.

* Now, let's see if we can do better by varying some other KNN options as well - in a *grid search*.

In [ ]:
param_grid = {'n_neighbors': np.array([1,2,4,8,16,32,64]),
                  'weights': ['uniform','distance'],
                       'p' : np.array([1,2])}

np.set_printoptions(suppress=True)
print(param_grid)

In [ ]:
from sklearn.grid_search import GridSearchCV
all_the_KNNs = GridSearchCV(KNN, param_grid, verbose=3)

A `GridSearchCV` object behaves just like a model:

In [ ]:
all_the_KNNs.fit(X_train, y_train)

In [ ]:
all_the_KNNs.score(X_test, y_test)

In [ ]:
all_the_KNNs.best_params_

<img src="../../scikit-learn/figures/grid_search_cross_validation.svg" width=100%>

BUT: what we want is a cross-validated model - so we need to pass our `GridSearch` to the cross validation score calculator to see the best we can do.

This will take a few moments, as the grid search is carried out for each CV fold...

In [ ]:
from sklearn.cross_validation import cross_val_score

R2 = cross_val_score(all_the_KNNs, X_scaled, y, cv=shuffle_split, scoring='r2')

In [ ]:
meanR2,errR2 = np.mean(R2),np.std(R2)
print("Mean score:",meanR2,"+/-",errR2)

In [ ]:
print(all_the_KNNs)

In [ ]:
KNN = neighbors.KNeighborsRegressor(n_neighbors=16,p=1,weights='distance')
KNN.fit(X_train,y_train)
y_knn_pred = KNN.predict(X_test)
mse = mean_squared_error(y_test,y_knn_pred)
print("MSE (KNN) =", mse)
print("R2 score (KNN) =",KNN.score(X_test, y_test))

In [ ]:
plot(y_test, y_knn_pred - y_test,'o',alpha=0.2)
title("k-NN Residuals - MSE = %.2f" % mse)
xlabel("Spectroscopic Redshift")
ylabel("Residual")
hlines(0,min(y_test),max(y_test),color="red")

## Quasar Classification with Random Forests


* Let's do a 3-class classification problem: star, galaxy, or QSO.


* A very good general-purpose classification (and regression!) algorithm is Random Forest. See [this blog post](http://blog.yhathq.com/posts/random-forests-in-python.html) for a nice high level introduction.


* ["A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset, and uses averaging to improve the predictive accuracy and control over-fitting.](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)


* Let's read in equal numbers of all three types of data, clean them up, and set $y$ equal to the classification label.

In [ ]:
all_sources = pd.read_csv("data/qso10000.csv",index_col=0,usecols=["objid","dered_r","u_g_color",\
                                                "g_r_color","r_i_color","i_z_color","diff_u",\
                                                "diff_g1","diff_i","diff_z","class"])[:1000]

all_sources = all_sources.append(pd.read_csv("data/star1000.csv",index_col=0,usecols=["objid","dered_r","u_g_color",\
                                                "g_r_color","r_i_color","i_z_color","diff_u",\
                                                "diff_g1","diff_i","diff_z","class"]))

all_sources = all_sources.append(pd.read_csv("data/galaxy1000.csv",index_col=0,usecols=["objid","dered_r","u_g_color",\
                                                "g_r_color","r_i_color","i_z_color","diff_u",\
                                                "diff_g1","diff_i","diff_z","class"]))

all_sources = all_sources[(all_sources["dered_r"] > -9999) & (all_sources["g_r_color"] > -10) & (all_sources["g_r_color"] < 10)]

all_labels = all_sources["class"]

all_features = copy.copy(all_sources)
del all_features["class"]

X = copy.copy(all_features.values)
y = copy.copy(all_labels.values)

In [ ]:
all_labels.tail()

In [ ]:
print("Feature vector shape =", X.shape)
print("Class label vector shape =", y.shape)

It might be more convenient to have our class labels be integers:

In [ ]:
# y[y=="QSO"] = 0
# y[y=="STAR"] = 1
# y[y=="GALAXY"] = 2

OK - let's turn on the machine learning:

** Random Forest Classification**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200,oob_score=True)
clf.fit(X,y)

What are the important features in the data?

In [ ]:
sorted(zip(all_sources.columns.values,clf.feature_importances_),key=lambda q: q[1],reverse=True)

In [ ]:
clf.oob_score_  ## "Out of Bag" Error

### Classifier improvement with GridSearchCV

In [ ]:
# Instantiate a classifier:
sdss_rf = RandomForestClassifier(n_estimators=200,oob_score=True)

# Parameter to search:
parameters = {'n_estimators':(10,50,200),"max_features": ["auto",3,5],
              'criterion':["gini","entropy"],"min_samples_leaf": [1,2]}

# Initial training/test split:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
# Do a grid search to find the highest 3-fold CV score:
rf_tune = GridSearchCV(sdss_rf, parameters, cv=3, verbose=1)
rf_opt = rf_tune.fit(X_train, y_train)

# Print the best score and estimator
print(rf_opt.best_score_)
print(rf_opt.best_estimator_)

In [ ]:
rf_opt.get_params()

In [ ]:
y_pred = rf_tune.fit(X_train, y_train).predict(X_test)

In [ ]:
# Compute confusion matrix

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

plt.matshow(cm)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')